In [1]:
!python -V

Python 3.10.2


In [2]:
import random
import re
from typing import Dict, Tuple

In [3]:
sequence_a = "AGTCGACCAGTGGGTATA"
sequence_b = "GCAGTCGTACCAAGTGGG"
aligned_a  = "..AGTCG.ACCA.GTGGGTATA"
aligned_b  = "GCAGTCGTACCAAGTGGG...."

In [4]:
def score_sequences(a: str, b: str):
    assert len(a) == len(b)
    score = 0
    for i in range(len(a)):
        if a[i] == "." or b[i] == ".":
            score += 0.1
        elif a[i] == b[i]:
            score += 1
        else:
            score += 0
    return score / (len(a) * 1.1)

In [5]:
score = score_sequences(sequence_a, sequence_b)
print(score)

0.15151515151515152


In [6]:
score = score_sequences(aligned_a, aligned_b)
print(score)

0.6115702479338841


In [7]:
def append_end(x: str, insert_char: str = "."):
    return x + insert_char

def append_start(x: str, insert_char: str = "."):
    return insert_char + x

def insert_random(x: str, insert_char: str = "."):
    random_index = random.randint(0, len(x)-1)
    return x[:random_index] + insert_char + x[random_index:]

def remove_random_space(x: str, insert_char: str = "\."):
    indexes = [y.start() for y in re.finditer(insert_char, x)]
    if not indexes:
        return x
    random_index = random.choice(indexes)
    return x[:random_index] + x[random_index+1:]

def trim_extra_gaps(a, b, insert_char: str = "."):
    new_a = ""
    new_b = ""
    for i in range(len(a)):
        if a[i] == insert_char and b[i] == insert_char:
            pass
        else:
            new_a += a[i]
            new_b += b[i]
    return new_a, new_b

In [8]:
# s = "aa"
# print(append_end(s))
# print(append_start(s))
# print(insert_random(s))
print(remove_random_space("a.b"))

ab


In [9]:
a = sequence_a
b = sequence_b

i = 0
while i < 1000000:
    i += 1
    score = score_sequences(a, b)
    # if random.random() > 0.5:
        # add char
    funcs = [append_start, append_end, insert_random]
    a_new = random.choice(funcs)(a)
    b_new = random.choice(funcs)(b)
    # else:
    #     # remove char
    #     a_new = remove_random_space(a)
    #     b_new = remove_random_space(b)

    new_score = score_sequences(a_new, b_new)
    if new_score > score:
        print("score increased from {} to {}".format(score, new_score))
        a = a_new
        b = b_new
    # if new_score >= 1.0:
    #     break
    # if i % 250 == 0:
    #     a, b = trim_extra_gaps(a, b)

a, b = trim_extra_gaps(a, b)
print(a)
print(b)


score increased from 0.15151515151515152 to 0.15311004784688995
score increased from 0.15311004784688995 to 0.2909090909090909
score increased from 0.2909090909090909 to 0.36796536796536794
score increased from 0.36796536796536794 to 0.40082644628099157
score increased from 0.40082644628099157 to 0.43083003952569154
score increased from 0.43083003952569154 to 0.4583333333333332
score increased from 0.4583333333333332 to 0.4763636363636362
..AGTCGA.CC.AGTGGGTATA
GCAGTCGTACCAAGTG.G.G..


In [10]:
class MatchPenalties:
    def __init__(
        self, 
        match: int = 12,
        mismatch: int = -6,
        gap: int = -10,
    ): 
        self.match = match
        self.mismatch = mismatch
        self.gap = gap

In [11]:
class Node:
    def __init__(
        self,
        match_type: int,
        t_max: int,
        t_min: int,
        prs: int,
        score: int,
        p1: int,
        p2: int
    ):
        self.match_type = match_type
        self.t_max = t_max
        self.t_min = t_min
        self.prs = prs
        self.score = score
        self.p1 = p1
        self.p2 = p2

    def __str__(self):
        return str(self.__dict__)

    def __repr__(self):
        return self.__str__()

In [12]:
def calc_f_score(x1: int, x2: int, match_mismatch: int, gap: int):
    if x2 < x1:
        return x2 * (match_mismatch + gap) * (x1 - x2)
    return x1 * (match_mismatch + gap) * (x2 - x1)


def calc_score(s1: str, s2: str, p1: int, p2: int, match_type: int, match_penalties: MatchPenalties):
    if match_type == 1:
        if s1[p1] == s2[p2]:
            return match_penalties.match
        return match_penalties.mismatch
    return match_penalties.gap

In [28]:
def process_node(s1: str, s2: str, p1: int, p2: int, match_type: int, prs: int, match_penalties: MatchPenalties):
    m = len(s1)
    n = len(s2)
    x1 = n - p2
    x2 = m - p1
    return Node(
        match_type=match_type, 
        t_max=prs+calc_f_score(x1, x2, match_penalties.match, match_penalties.gap), 
        t_min=prs+calc_f_score(x1, x2, match_penalties.mismatch, match_penalties.gap),
        prs=prs,
        score=prs+calc_score(s1, s2, p1, p2, match_type, match_penalties),
        p1=p1,
        p2=p2,
    )

In [32]:
def solve_fogsaa(s1: str, s2: str, match_penalties: MatchPenalties = MatchPenalties()):
    p1 = 0
    p2 = 0
    
    history: Dict[Tuple(int, int), Node] = {}

    prs = 0

    history[(p1, p2)] = match_ab = process_node(s1, s2, p1, p2, match_type=1, prs=prs, match_penalties=match_penalties)
    history[(p1+1, p2)] = match_Ab = process_node(s1, s2, p1+1, p2, match_type=2, prs=prs, match_penalties=match_penalties)
    history[(p1, p2+1)] = match_aB = process_node(s1, s2, p1, p2+1, match_type=3, prs=prs, match_penalties=match_penalties)

    current_node = sorted([match_ab, match_Ab, match_aB], key=lambda x: x.score, reverse=True)[0]
    print(current_node)
    p1 = current_node.p1
    p2 = current_node.p2
    prs = current_node.score

    while True:
        history[(p1+1, p2+1)] = match_ab = process_node(s1, s2, p1+1, p2+1, match_type=1, prs=prs, match_penalties=match_penalties)
        history[(p1+1, p2)] = match_Ab = process_node(s1, s2, p1+1, p2, match_type=2, prs=prs, match_penalties=match_penalties)
        history[(p1, p2+1)] = match_aB = process_node(s1, s2, p1, p2+1, match_type=3, prs=prs, match_penalties=match_penalties)
        new_best = sorted([match_ab, match_Ab, match_aB], key=lambda x: x.score, reverse=True)[0]
        
        current_node = new_best
        print(current_node)
        
        p1 = current_node.p1
        p2 = current_node.p2
        prs = current_node.score

        if p1 >= len(s1)-1 or p2 >= len(s2)-1:
            break

    print(history)

solve_fogsaa("abcdefg", "abcdefg")

{'match_type': 2, 't_max': 12, 't_min': -96, 'score': -10, 'p1': 1, 'p2': 0}
{'match_type': 2, 't_max': 10, 't_min': -170, 'score': -20, 'p1': 2, 'p2': 0}
{'match_type': 2, 't_max': 4, 't_min': -212, 'score': -30, 'p1': 3, 'p2': 0}
{'match_type': 2, 't_max': -6, 't_min': -222, 'score': -40, 'p1': 4, 'p2': 0}
{'match_type': 2, 't_max': -20, 't_min': -200, 'score': -50, 'p1': 5, 'p2': 0}
{'match_type': 3, 't_max': -34, 't_min': -178, 'score': -60, 'p1': 5, 'p2': 1}
{'match_type': 3, 't_max': -48, 't_min': -156, 'score': -70, 'p1': 5, 'p2': 2}
{'match_type': 2, 't_max': -62, 't_min': -134, 'score': -80, 'p1': 6, 'p2': 2}
{(0, 0): {'match_type': 1, 't_max': 0, 't_min': 0, 'score': 12, 'p1': 0, 'p2': 0}, (1, 0): {'match_type': 2, 't_max': 12, 't_min': -96, 'score': -10, 'p1': 1, 'p2': 0}, (0, 1): {'match_type': 3, 't_max': 12, 't_min': -96, 'score': -10, 'p1': 0, 'p2': 1}, (2, 1): {'match_type': 3, 't_max': -10, 't_min': -100, 'score': -30, 'p1': 2, 'p2': 1}, (2, 0): {'match_type': 2, 't_ma